In [1]:
import numpy as np
import pandas as pd
import seaborn as sea
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import lightgbm

import warnings
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv("/kaggle/input/WiDSWorldWide_GlobalDathon26/train.csv")
test = pd.read_csv("/kaggle/input/WiDSWorldWide_GlobalDathon26/test.csv")

train.head()

,event_id,num_perimeters_0_5h,dt_first_last_0_5h,low_temporal_resolution_0_5h,area_first_ha,area_growth_abs_0_5h,area_growth_rel_0_5h,area_growth_rate_ha_per_h,log1p_area_first,log1p_growth,...,dist_fit_r2_0_5h,alignment_cos,alignment_abs,cross_track_component,along_track_speed,event_start_hour,event_start_dayofweek,event_start_month,time_to_hit_hours,event
0,10892457,3,4.265188,0,79.696304,2.875935,0.036086,0.674281,4.390693,1.354787,...,0.886373,-0.054649,0.054649,-1.937219,-0.106026,19,4,5,18.892512,0
1,11757157,2,1.169918,0,8.946749,0.000000,0.000000,0.000000,2.297246,0.000000,...,0.000000,-0.568898,0.568898,-0.000000,-0.000000,4,4,6,22.048108,1
2,11945086,4,4.777526,0,106.482638,0.000000,0.000000,0.000000,4.677329,0.000000,...,0.000000,0.882385,0.882385,0.000000,0.000000,22,4,8,0.888895,1
3,12044083,1,0.000000,1,67.631125,0.000000,0.000000,0.000000,4.228746,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,20,5,8,60.953021,0
4,12052347,2,4.975273,0,35.632874,0.000000,0.000000,0.000000,3.600946,0.000000,...,0.000000,0.934634,0.934634,-0.000000,0.000000,21,5,7,44.990274,0


# Dataset Preparations

In [3]:
HORIZONS = [12, 24, 48, 72]

In [4]:
def build_horizon_targets(df:pd.DataFrame, horizons=HORIZONS):
    ## Ensures the required columns are present
    required = {"event_id", "time_to_hit_hours", "event"}
    missing = required - set(df.columns)
    if missing: raise KeyError(f"Train is missing required columns: {sorted(missing)}")
    out = df.copy()

    ## Convert event-related features to numeric type
    out["event_id"] = pd.to_numeric(out["event_id"], errors="raise").astype(np.int64)
    out["event"]    = pd.to_numeric(out["event"], errors="raise").astype(np.int64)
    out["time_to_hit_hours"] = pd.to_numeric(out["time_to_hit_hours"], errors="raise").astype(float)

    ## Extract time and event state
    times  = out["time_to_hit_hours"].values
    events = out["event"].values

    for horizon in horizons:
        ## Label unknown states and known states for target labeling
        unknown =  (events == 0) & (times < horizon)
        y       = ((events == 1) & (times <= horizon)).astype(float)
        y[unknown] = np.nan
        out[f"prob_{horizon}h"] = y
        out[f"mask_{horizon}h"] = (~unknown).astype(np.int8)

    ## Raise errors for duplicates in event_id
    if out["event_id"].duplicated().any():
        dup = out.loc[out["event_id"].duplicated(), "event_id"].iloc[0]
        raise ValueError(f"Duplicate event_id found in train: {dup}")

    ## Checks for monotonicity in probabilities of events
    for a, b in zip(horizons[:-1], horizons[1:]):
        ma = out[f"mask_{a}h"].values.astype(bool)
        mb = out[f"mask_{b}h"].values.astype(bool)
        both = ma & mb

        ya = out.loc[both, f"prob_{a}h"].values
        yb = out.loc[both, f"prob_{b}h"].values

        non_monotonic_pairs = np.where((~np.isnan(ya)) & (~np.isnan(yb)) & (ya > yb))[0]
        if non_monotonic_pairs.size:
            i = out.index[both][bad[0]]
            raise ValueError(
                f"Monotonicity violated in labels at row index {i}: "
                f"prob_{a}h={out.at[i, f'prob_{a}h']}, prob_{b}h={out.at[i, f'prob_{b}h']}"
            )
    return out





def organize_test(df: pd.DataFrame, horizons=HORIZONS):
    out = df.copy()
    ## Ensures event_id is a column in test data
    if "event_id" not in out.columns: raise KeyError("Test is missing required column: event_id")
    ## Converts event_id column to num type
    out["event_id"] = pd.to_numeric(out["event_id"], errors="raise").astype(np.int64)
    ## Ensures no event_id duplicates 
    if out["event_id"].duplicated().any():
        dup = out.loc[out["event_id"].duplicated(), "event_id"].iloc[0]
        raise ValueError(f"Duplicate event_id found in test: {dup}")

    for horizon in horizons:
        out[f"prob_{horizon}h"] = np.nan
    return out

In [5]:
## Build target features
train_organized = build_horizon_targets(train)
test_organized  = organize_test(test)

## Extract labels and masks
label_cols = [f"prob_{horizon}h" for horizon in HORIZONS]
mask_cols  = [f"mask_{horizon}h" for horizon in HORIZONS]

## Organize the column orders in datasets
train_cols_order = (
    ["event_id"]
    + [c for c in train_organized.columns if c not in (["event_id"] + label_cols + mask_cols)]
    + label_cols
    + mask_cols
)
test_cols_order = (
    ["event_id"]
    + [c for c in test_organized.columns if c not in (["event_id"] + label_cols)]
    + label_cols
)
train_organized = train_organized[train_cols_order]
test_organized  = test_organized[test_cols_order]

## Save datasets
train_organized.to_csv("train_organized.csv", index=False)
test_organized.to_csv("test_organized.csv", index=False)

print("Saved: train_organized.csv")
print("Saved: test_organized.csv")
print("Train shape:", train_organized.shape)
print("Test shape:", test_organized.shape)
print("Train label non-null counts:", {c: int(train_organized[c].notna().sum()) for c in label_cols})

Saved: train_organized.csv
Saved: test_organized.csv
Train shape: (221, 45)
Test shape: (95, 39)
Train label non-null counts: {'prob_12h': 215, 'prob_24h': 196, 'prob_48h': 166, 'prob_72h': 69}


In [6]:
train = pd.read_csv("train_organized.csv")
test = pd.read_csv("test_organized.csv")

## Filter out features for training
exclude_cols = {"event_id", "time_to_hit_hours", "event", *label_cols, *mask_cols}
feature_cols = [c for c in train.columns if c not in exclude_cols]

## Convert features to numeric type
for feature in feature_cols:
    train[feature] = pd.to_numeric(train[feature], errors="coerce")
    if feature in test.columns:
        test[feature] = pd.to_numeric(test[feature], errors="coerce")


## Creates train and test sets
X_train = np.nan_to_num(train[feature_cols].astype(float).values, nan=0.0, posinf=0.0, neginf=0.0)
X_test  = np.nan_to_num(test[feature_cols].astype(float).values, nan=0.0, posinf=0.0, neginf=0.0)

## Scales train and test sets
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled  = scaler.transform(X_test)

## Save the scaled train and test sets
train_scaled = train.copy()
test_scaled = test.copy()

train_scaled[feature_cols] = X_train_scaled
test_scaled[feature_cols]  = X_test_scaled
train_scaled.to_csv("train_scaled.csv", index=False)
test_scaled.to_csv("test_scaled.csv", index=False)

print("Saved: train_scaled.csv")
print("Saved: test_scaled.csv")
print("Scaled feature count:", len(feature_cols))

Saved: train_scaled.csv
Saved: test_scaled.csv
Scaled feature count: 34


# Metric Creation

In [7]:
def _to_numpy(x):
    if isinstance(x, np.ndarray): return x
    return np.asarray(x)

In [8]:
def harrell_c_index(time_to_hit_hours, event, risk_score):
    t = _to_numpy(time_to_hit_hours).astype(float)
    e = _to_numpy(event).astype(int)
    r = _to_numpy(risk_score).astype(float)

    n = t.shape[0]
    conc = 0.0
    ties = 0.0
    comparable = 0.0

    for i in range(n):
        if e[i] != 1:
            continue
        ti = t[i]
        ri = r[i]
        for j in range(n):
            if j == i:
                continue
            tj = t[j]
            if ti < tj:
                comparable += 1.0
                rj = r[j]
                if ri > rj:
                    conc += 1.0
                elif ri == rj:
                    ties += 1.0

    if comparable == 0.0:
        return 0.5
    return (conc + 0.5 * ties) / comparable

In [9]:
def brier_at_horizon(time_to_hit_hours, event, prob_hit_by_h, H):
    t = _to_numpy(time_to_hit_hours).astype(float)
    e = _to_numpy(event).astype(int)
    p = _to_numpy(prob_hit_by_h).astype(float)

    valid = ~((e == 0) & (t < H))
    if not np.any(valid): return 0.25

    hit_by_h = ((e == 1) & (t <= H)).astype(float)
    y  = hit_by_h[valid]
    pv = np.clip(p[valid], 0.0, 1.0)
    return float(np.mean((pv - y) ** 2))


def brier_at_horizon(time_to_hit_hours, event, prob_hit_by_h, H):
    t = _to_numpy(time_to_hit_hours).astype(float)
    e = _to_numpy(event).astype(int)
    p = _to_numpy(prob_hit_by_h).astype(float)

    valid = ~((e == 0) & (t < H))
    if not np.any(valid):
        return 0.25

    hit_by_h = ((e == 1) & (t <= H)).astype(float)
    y = hit_by_h[valid]
    pv = p[valid]

    pv = np.clip(pv, 0.0, 1.0)
    return float(np.mean((pv - y) ** 2))

def weighted_brier(time_to_hit_hours, event, prob_24h, prob_48h, prob_72h):
    b24 = brier_at_horizon(time_to_hit_hours, event, prob_24h, 24.0)
    b48 = brier_at_horizon(time_to_hit_hours, event, prob_48h, 48.0)
    b72 = brier_at_horizon(time_to_hit_hours, event, prob_72h, 72.0)
    return 0.3 * b24 + 0.4 * b48 + 0.3 * b72

def hybrid_score(time_to_hit_hours, event, prob_24h, prob_48h, prob_72h, risk_score):
    c = harrell_c_index(time_to_hit_hours, event, risk_score)
    wb = weighted_brier(time_to_hit_hours, event, prob_24h, prob_48h, prob_72h)
    return 0.3 * c + 0.7 * (1.0 - wb)

def make_hybrid_callback(time_to_hit_hours, event, get_probs_and_risk, period=10, name="hybrid"):
    t = _to_numpy(time_to_hit_hours).astype(float)
    e = _to_numpy(event).astype(int)

    def _cb(env):
        it = int(getattr(env, "iteration", 0))
        if it % int(period) != 0 and it != 0:
            return
        out = get_probs_and_risk(env)
        p24 = _to_numpy(out["prob_24h"]).astype(float)
        p48 = _to_numpy(out["prob_48h"]).astype(float)
        p72 = _to_numpy(out["prob_72h"]).astype(float)
        rs  = _to_numpy(out["risk_score"]).astype(float)
        val = hybrid_score(t, e, p24, p48, p72, rs)
        print(f"[{name}] iter={it}  value={val:.6f}")

    return _cb

def make_brier_feval(H):
    H = float(H)
    def _feval(preds, dataset):
        t = _to_numpy(dataset.get_field("time_to_hit_hours")).astype(float)
        e = _to_numpy(dataset.get_label()).astype(int)
        p = _to_numpy(preds).astype(float)
        b = brier_at_horizon(t, e, p, H)
        return (f"brier_{int(H)}h", b, False)
    return _feval

In [10]:
def monotone_probs(p12, p24, p48, p72):
    p12 = np.clip(p12, 0.0, 1.0)
    p24 = np.clip(p24, 0.0, 1.0)
    p48 = np.clip(p48, 0.0, 1.0)
    p72 = np.clip(p72, 0.0, 1.0)
    p24 = np.maximum(p24, p12)
    p48 = np.maximum(p48, p24)
    p72 = np.maximum(p72, p48)
    return p12, p24, p48, p72


# Model Evaluation

In [11]:
train = pd.read_csv("train_scaled.csv")
test = pd.read_csv("test_scaled.csv")

X_all  = train[feature_cols].astype(float).values
X_test = test[feature_cols].astype(float).values
t_all  = train["time_to_hit_hours"].astype(float).values
e_all  = train["event"].astype(int).values

## Split into train and validation sets
train_idx, val_idx = train_test_split(np.arange(len(train)), test_size=0.2, random_state=3126, stratify=e_all)
X_train, X_val = X_all[train_idx], X_all[val_idx]
t_val, e_val   = t_all[val_idx], e_all[val_idx]

In [12]:
def make_dataset_for_h(h):
    y = train[f"prob_{h}h"].values
    m = ~np.isnan(y)
    m_tr = m[train_idx]
    m_va = m[val_idx]
    X_tr = X_all[train_idx][m_tr]
    y_tr = y[train_idx][m_tr].astype(float)
    X_va = X_all[val_idx][m_va]
    y_va = y[val_idx][m_va].astype(float)
    dtr = lightgbm.Dataset(X_tr, label=y_tr, feature_name=feature_cols, free_raw_data=False)
    dva = lightgbm.Dataset(X_va, label=y_va, feature_name=feature_cols, free_raw_data=False)
    return dtr, dva, m_va

In [13]:
base_params = {
    "objective": "binary",
    "learning_rate": 0.043,
    "num_leaves": 42,
    "min_data_in_leaf": 32,
    "feature_fraction": 0.73,
    "bagging_fraction": 0.86,
    "max_depth":2,
    "bagging_freq": 2,
    "lambda_l2": 0.0,
    "lamda_l1":0.0,
    "verbosity": -1,
    "seed": 42,
    "force_col_wise": True,
}

In [14]:
dtr12, dva12, mva12 = make_dataset_for_h(12)
dtr24, dva24, mva24 = make_dataset_for_h(24)
dtr48, dva48, mva48 = make_dataset_for_h(48)
dtr72, dva72, mva72 = make_dataset_for_h(72)

b12 = lightgbm.Booster(params=base_params, train_set=dtr12)
b24 = lightgbm.Booster(params=base_params, train_set=dtr24)
b48 = lightgbm.Booster(params=base_params, train_set=dtr48)
b72 = lightgbm.Booster(params=base_params, train_set=dtr72)

max_rounds  = 5000
patience    = 200
print_every = 10
best_iter   = 0
best_score  = -1e18
since_best  = 0
best_models = None

for it in range(1, max_rounds + 1):
    b12.update()
    b24.update()
    b48.update()
    b72.update()

    if it % print_every == 0 or it == 1:
        p12 = b12.predict(X_val, num_iteration=it)
        p24 = b24.predict(X_val, num_iteration=it)
        p48 = b48.predict(X_val, num_iteration=it)
        p72 = b72.predict(X_val, num_iteration=it)
        p12, p24, p48, p72 = monotone_probs(p12, p24, p48, p72)

        risk = 0.3 * p24 + 0.4 * p48 + 0.3 * p72
        score = hybrid_score(t_val, e_val, p24, p48, p72, risk)

        brier24 = brier_at_horizon(t_val, e_val, p24, 24.0)
        brier48 = brier_at_horizon(t_val, e_val, p48, 48.0)
        brier72 = brier_at_horizon(t_val, e_val, p72, 72.0)
        cidx = harrell_c_index(t_val, e_val, risk)

        print(f"iter={it}  hybrid={score:.6f}  cindex={cidx:.6f}  wbrier={0.3*brier24+0.4*brier48+0.3*brier72:.6f}  b24={brier24:.6f}  b48={brier48:.6f}  b72={brier72:.6f}")

        if score > best_score + 1e-12:
            best_score = score
            best_iter = it
            since_best = 0
            best_models = (b12.model_to_string(num_iteration=it),
                           b24.model_to_string(num_iteration=it),
                           b48.model_to_string(num_iteration=it),
                           b72.model_to_string(num_iteration=it))
        else:
            since_best += print_every
            if since_best >= patience:
                break

if best_models is None:
    raise RuntimeError("No best model snapshot captured.")

b12_best = lightgbm.Booster(model_str=best_models[0])
b24_best = lightgbm.Booster(model_str=best_models[1])
b48_best = lightgbm.Booster(model_str=best_models[2])
b72_best = lightgbm.Booster(model_str=best_models[3])

print(f"Best iter={best_iter}  best_hybrid={best_score:.6f}")

def train_full(h):
    y = train[f"prob_{h}h"].values
    m = ~np.isnan(y)
    X = X_all[m]
    yy = y[m].astype(float)
    dtr = lightgbm.Dataset(X, label=yy, feature_name=feature_cols, free_raw_data=False)
    booster = lightgbm.Booster(params=base_params, train_set=dtr)
    for _ in range(best_iter):
        booster.update()
    return booster

b12_full = train_full(12)
b24_full = train_full(24)
b48_full = train_full(48)
b72_full = train_full(72)

p12 = b12_full.predict(X_test)
p24 = b24_full.predict(X_test)
p48 = b48_full.predict(X_test)
p72 = b72_full.predict(X_test)
p12, p24, p48, p72 = monotone_probs(p12, p24, p48, p72)

iter=1  hybrid=0.870333  cindex=0.915479  wbrier=0.149015  b24=0.201890  b48=0.221120  b72=0.000000
iter=10  hybrid=0.917864  cindex=0.932790  wbrier=0.088533  b24=0.120368  b48=0.131057  b72=0.000000
iter=20  hybrid=0.939219  cindex=0.933809  wbrier=0.058462  b24=0.079758  b48=0.086336  b72=0.000000
iter=30  hybrid=0.946389  cindex=0.935845  wbrier=0.049092  b24=0.066973  b48=0.072499  b72=0.000000
iter=40  hybrid=0.952957  cindex=0.935845  wbrier=0.039709  b24=0.053399  b48=0.059223  b72=0.000000
iter=50  hybrid=0.956374  cindex=0.935845  wbrier=0.034827  b24=0.046210  b48=0.052411  b72=0.000000
iter=60  hybrid=0.958030  cindex=0.938900  wbrier=0.033772  b24=0.044308  b48=0.051199  b72=0.000000
iter=70  hybrid=0.959194  cindex=0.938900  wbrier=0.032109  b24=0.041598  b48=0.049074  b72=0.000000
iter=80  hybrid=0.959819  cindex=0.938900  wbrier=0.031215  b24=0.040031  b48=0.048014  b72=0.000000
iter=90  hybrid=0.962371  cindex=0.946029  wbrier=0.030626  b24=0.038905  b48=0.047385  b72=

In [15]:
sub = pd.DataFrame({
    "event_id": test["event_id"].astype(np.int64).values,
    "prob_12h": p12.astype(float),
    "prob_24h": p24.astype(float),
    "prob_48h": p48.astype(float),
    "prob_72h": p72.astype(float),
})

sub.to_csv("submission.csv", index=False)
print("Saved: submission.csv")
print(sub.head())

Saved: submission.csv
   event_id  prob_12h  prob_24h  prob_48h  prob_72h
0  10662602  0.000459  0.002662  0.002662       1.0
1  13353600  0.461859  0.999224  0.999224       1.0
2  13942327  0.000142  0.000568  0.000568       1.0
3  16112781  0.808606  0.952079  0.999317       1.0
4  17132808  0.029147  0.029147  0.029147       1.0
